# * VINSIGHT : Revenue

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Revenue

In [2]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000100GEO' --Total Revenue (Geo)
        , 'DB0R000100GEO' --Total Revenue (Geo) : DTAC
        , 'TB0R000100GEO' --Total Revenue (Geo) : TRUE

        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B0R000101GEO' --Mobile Revenue (Geo)
        , 'DB0R000101GEO' --Mobile Revenue (Geo) : DTAC
        , 'TB0R000101GEO' --Mobile Revenue (Geo) : TMH
        
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'B2R020100' --Postpaid Revenue B2B
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'DB2R020100' --Postpaid Revenue B2B : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'TB2R020100' --Postpaid Revenue B2B : TMH
        
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
        
        , 'TB3R000100' --TOL Revenue
        
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-28, 13:10:36
   -> v_start_date: 20250101


In [3]:
''' Automate Month Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

In [4]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display

In [5]:
# ''' Sample Data '''

# sample_mth_df = monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]
# sample_mth_df

### Special Metric

#### Revenue Corporate (C)
    B0R000100	Total Revenue
    DB0R000100	Total Revenue : DTAC
    TB0R000100	Total Revenue : TRUE
    
    B0R000101	Mobile Revenue
    DB0R000101	Mobile Revenue : DTAC
    TB0R000101	Mobile Revenue : TMH

In [6]:
''' B0R000100 : Total Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'TB1R000100', 'DB2R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00147')

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_df = total_rev_corp_df.loc[total_rev_corp_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_corp_df = total_rev_corp_df.loc[total_rev_corp_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_corp_df['TOTAL'] = np.where(total_rev_corp_df['METRIC_CD']=='B0R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TOL'] = np.where(total_rev_corp_df['METRIC_CD']=='TB3R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TVS'] = np.where(total_rev_corp_df['METRIC_CD']=='TB4R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['CHK_SUM'] = np.where(total_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100','TB3R000100','TB4R000100']), total_rev_corp_df['C'], 0)

total_rev_corp_df = total_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_df['CHK_DIFF'] = total_rev_corp_df['TOTAL'] - total_rev_corp_df['CHK_SUM']
total_rev_corp_df = total_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_df = total_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_df[col] = total_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_T,POST_D,TOL,TVS
0,202501,2025-05-28 02:04:38,"-971,176,946","12,770,106,773","11,798,929,827","2,444,926,240","1,839,813,343","3,731,079,534","2,739,914,585","1,597,056,733","417,316,338"
1,202502,2025-05-28 02:04:38,"-1,325,713,577","12,236,601,216","10,910,887,639","2,217,259,007","1,648,646,062","3,726,430,939","2,727,363,466","1,609,673,918","307,227,824"
2,202503,2025-05-28 02:04:38,"-973,748,116","12,516,306,281","11,542,558,166","2,466,370,982","1,775,352,571","3,708,553,920","2,734,968,120","1,616,442,724","214,617,965"
3,202504,2025-05-28 02:04:38,"-3,767,633,755","12,526,105,797","8,758,472,043","2,366,358,116","1,713,487,323","3,767,633,755","2,744,630,192","1,616,821,911","317,174,500"
4,202505,2025-05-28 02:04:38,"-2,941,399,529","10,347,356,403","7,405,956,874","2,042,611,277","1,496,661,018","2,941,399,529","2,239,192,171","1,439,967,291","187,525,117"


In [7]:
''' TB0R000100 : Total Revenue : TRUE '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100')

v_metric_list = [
    'TB0R000100' #Total Revenue : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_corp_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_true_df = total_rev_corp_true_df.loc[total_rev_corp_true_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_corp_true_df = total_rev_corp_true_df.loc[total_rev_corp_true_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_corp_true_df['TOTAL_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB0R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['PRE_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB1R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['POST_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB2R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TOL'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB3R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TVS'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB4R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['CHK_SUM'] = np.where(total_rev_corp_true_df['METRIC_CD'].isin(['TB1R000100','TB2R000100','TB3R000100','TB4R000100']), total_rev_corp_true_df['C'], 0)

total_rev_corp_true_df = total_rev_corp_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_true_df['CHK_DIFF'] = total_rev_corp_true_df['TOTAL_T'] - total_rev_corp_true_df['CHK_SUM']
total_rev_corp_true_df = total_rev_corp_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_true_df = total_rev_corp_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_T', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_true_df[col] = total_rev_corp_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_T,TOL,TVS
0,202501,2025-05-28 02:04:38,"-3,731,079,534","8,190,378,845","4,459,299,311","2,444,926,240","3,731,079,534","1,597,056,733","417,316,338"
1,202502,2025-05-28 02:04:38,"-3,726,430,939","7,860,591,688","4,134,160,749","2,217,259,007","3,726,430,939","1,609,673,918","307,227,824"
2,202503,2025-05-28 02:04:38,"-3,708,553,920","8,005,985,591","4,297,431,671","2,466,370,982","3,708,553,920","1,616,442,724","214,617,965"
3,202504,2025-05-28 02:04:38,"-3,767,633,755","8,067,988,282","4,300,354,528","2,366,358,116","3,767,633,755","1,616,821,911","317,174,500"
4,202505,2025-05-28 02:04:38,"-2,941,399,529","6,611,503,215","3,670,103,686","2,042,611,277","2,941,399,529","1,439,967,291","187,525,117"


In [8]:
''' DB0R000100 : Total Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'DB2R000100')

v_metric_list = [
    'DB0R000100' #Total Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_dtac_df = total_rev_corp_dtac_df.loc[total_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_corp_dtac_df = total_rev_corp_dtac_df.loc[total_rev_corp_dtac_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_corp_dtac_df['TOTAL_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB0R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['PRE_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['POST_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['CHK_SUM'] = np.where(total_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), total_rev_corp_dtac_df['C'], 0)

total_rev_corp_dtac_df = total_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
total_rev_corp_dtac_df['CHK_DIFF'] = total_rev_corp_dtac_df['TOTAL_D'] - total_rev_corp_dtac_df['CHK_SUM']
total_rev_corp_dtac_df = total_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_dtac_df = total_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_D']]

mod_col_list = total_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_dtac_df[col] = total_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_D
0,202501,2025-05-28 02:04:38,"2,759,902,588","4,579,727,928","7,339,630,516","1,839,813,343","2,739,914,585"
1,202502,2025-05-28 02:04:38,"2,400,717,361","4,376,009,529","6,776,726,890","1,648,646,062","2,727,363,466"
2,202503,2025-05-28 02:04:38,"2,734,805,804","4,510,320,691","7,245,126,495","1,775,352,571","2,734,968,120"
3,202504,2025-05-28 02:04:38,-0,"4,458,117,515","4,458,117,515","1,713,487,323","2,744,630,192"
4,202505,2025-05-28 02:04:38,-0,"3,735,853,188","3,735,853,188","1,496,661,018","2,239,192,171"


In [9]:
''' B0R000101 : Mobile Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'TB2R000100', 'DB1R000100', 'TB1R000100')

v_metric_list = [
    'B0R000101' #Mobile Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_df = mobile_rev_corp_df.loc[mobile_rev_corp_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_corp_df = mobile_rev_corp_df.loc[mobile_rev_corp_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_corp_df['MOBILE'] = np.where(mobile_rev_corp_df['METRIC_CD']=='B0R000101', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['CHK_SUM'] = np.where(mobile_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100']), mobile_rev_corp_df['C'], 0)

mobile_rev_corp_df = mobile_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_df['CHK_DIFF'] = mobile_rev_corp_df['MOBILE'] - mobile_rev_corp_df['CHK_SUM']
mobile_rev_corp_df = mobile_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_df = mobile_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D']]

mod_col_list = mobile_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_df[col] = mobile_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_T,POST_D
0,202501,2025-05-28 02:04:38,"-971,176,946","10,755,733,702","9,784,556,756","2,444,926,240","1,839,813,343","3,731,079,534","2,739,914,585"
1,202502,2025-05-28 02:04:38,"-1,325,713,577","10,319,699,474","8,993,985,897","2,217,259,007","1,648,646,062","3,726,430,939","2,727,363,466"
2,202503,2025-05-28 02:04:38,"-973,748,116","10,685,245,593","9,711,497,477","2,466,370,982","1,775,352,571","3,708,553,920","2,734,968,120"
3,202504,2025-05-28 02:04:38,"-3,767,633,755","10,592,109,386","6,824,475,632","2,366,358,116","1,713,487,323","3,767,633,755","2,744,630,192"
4,202505,2025-05-28 02:04:38,"-2,941,399,529","8,719,863,994","5,778,464,465","2,042,611,277","1,496,661,018","2,941,399,529","2,239,192,171"


In [10]:
''' TB0R000101 : Mobile Revenue : TMH '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB2R000100', 'TB1R000100')

v_metric_list = [
    'TB0R000101' #Mobile Revenue : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_corp_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.loc[mobile_rev_corp_tmh_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.loc[mobile_rev_corp_tmh_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_corp_tmh_df['MOBILE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB0R000101', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['PRE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['POST_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['CHK_SUM'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R000100']), mobile_rev_corp_tmh_df['C'], 0)

mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_tmh_df['CHK_DIFF'] = mobile_rev_corp_tmh_df['MOBILE_T'] - mobile_rev_corp_tmh_df['CHK_SUM']
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_T']]

mod_col_list = mobile_rev_corp_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_tmh_df[col] = mobile_rev_corp_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_T
0,202501,2025-05-28 02:04:38,"-3,731,079,534","6,176,005,774","2,444,926,240","2,444,926,240","3,731,079,534"
1,202502,2025-05-28 02:04:38,"-3,726,430,939","5,943,689,946","2,217,259,007","2,217,259,007","3,726,430,939"
2,202503,2025-05-28 02:04:38,"-3,708,553,920","6,174,924,902","2,466,370,982","2,466,370,982","3,708,553,920"
3,202504,2025-05-28 02:04:38,"-3,767,633,755","6,133,991,871","2,366,358,116","2,366,358,116","3,767,633,755"
4,202505,2025-05-28 02:04:38,"-2,941,399,529","4,984,010,806","2,042,611,277","2,042,611,277","2,941,399,529"


In [11]:
''' DB0R000101 : Mobile Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'DB1R000100')

v_metric_list = [
    'DB0R000101' #Mobile Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.loc[mobile_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.loc[mobile_rev_corp_dtac_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_corp_dtac_df['MOBILE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB0R000101', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['PRE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['POST_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['CHK_SUM'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), mobile_rev_corp_dtac_df['C'], 0)

mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_dtac_df['CHK_DIFF'] = mobile_rev_corp_dtac_df['MOBILE_D'] - mobile_rev_corp_dtac_df['CHK_SUM']
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_D']]

mod_col_list = mobile_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_dtac_df[col] = mobile_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_D
0,202501,2025-05-28 02:04:38,"2,759,902,588","4,579,727,928","7,339,630,516","1,839,813,343","2,739,914,585"
1,202502,2025-05-28 02:04:38,"2,400,717,361","4,376,009,529","6,776,726,890","1,648,646,062","2,727,363,466"
2,202503,2025-05-28 02:04:38,"2,734,805,804","4,510,320,691","7,245,126,495","1,775,352,571","2,734,968,120"
3,202504,2025-05-28 02:04:38,-0,"4,458,117,515","4,458,117,515","1,713,487,323","2,744,630,192"
4,202505,2025-05-28 02:04:38,0,"3,735,853,188","3,735,853,188","1,496,661,018","2,239,192,171"


#### Revenue Geo (P)
    B0R000100GEO	Total Revenue (Geo)
    DB0R000100GEO	Total Revenue (Geo) : DTAC
    TB0R000100GEO	Total Revenue (Geo) : TRUE
    
    B0R000101GEO	Mobile Revenue (Geo)
    DB0R000101GEO	Mobile Revenue (Geo) : DTAC
    TB0R000101GEO	Mobile Revenue (Geo) : TMH

In [12]:
''' B0R000100GEO : Total Revenue (Geo) '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_df = total_rev_geo_df.loc[total_rev_geo_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_geo_df = total_rev_geo_df.loc[total_rev_geo_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_geo_df['TOTAL'] = np.where(total_rev_geo_df['METRIC_CD']=='B0R000100GEO', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TOL'] = np.where(total_rev_geo_df['METRIC_CD']=='TB3R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TVS'] = np.where(total_rev_geo_df['METRIC_CD']=='TB4R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['CHK_SUM'] = np.where(total_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100','TB3R000100','TB4R000100']), total_rev_geo_df['P'], 0)

total_rev_geo_df = total_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_df['CHK_DIFF'] = total_rev_geo_df['TOTAL'] - total_rev_geo_df['CHK_SUM']
total_rev_geo_df = total_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_df = total_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_df[col] = total_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202501,2025-05-28 02:04:38,0,"12,034,347,430","12,034,347,430","2,440,530,082","1,840,026,982","3,336,770,490","2,531,625,556","1,592,511,720","292,882,600"
1,202502,2025-05-28 02:04:38,0,"11,619,121,062","11,619,121,062","2,211,389,140","1,648,818,151","3,333,288,223","2,527,190,353","1,605,419,142","293,016,052"
2,202503,2025-05-28 02:04:38,0,"11,995,980,677","11,995,980,677","2,459,164,272","1,775,434,050","3,319,489,222","2,532,397,179","1,612,256,745","297,239,208"
3,202504,2025-05-28 02:04:38,0,"11,891,585,207","11,891,585,207","2,359,595,474","1,713,499,280","3,371,023,854","2,530,919,910","1,612,742,089","303,804,601"
4,202505,2025-05-28 02:04:38,0,"9,868,224,191","9,868,224,191","2,029,638,601","1,496,666,317","2,675,453,980","2,054,898,708","1,434,529,527","177,037,058"


In [13]:
''' TB0R000100GEO : Total Revenue (Geo) : TRUE '''

v_metric_list = [
    'TB0R000100GEO' #Total Revenue (Geo) : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_geo_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_true_df = total_rev_geo_true_df.loc[total_rev_geo_true_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_geo_true_df = total_rev_geo_true_df.loc[total_rev_geo_true_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_geo_true_df['TOTAL_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB0R000100GEO', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['PRE_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB1R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['POST_B2C_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB2R010100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TOL'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB3R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TVS'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB4R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['CHK_SUM'] = np.where(total_rev_geo_true_df['METRIC_CD'].isin(['TB1R000100','TB2R010100','TB3R000100','TB4R000100']), total_rev_geo_true_df['P'], 0)

total_rev_geo_true_df = total_rev_geo_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_true_df['CHK_DIFF'] = total_rev_geo_true_df['TOTAL_T'] - total_rev_geo_true_df['CHK_SUM']
total_rev_geo_true_df = total_rev_geo_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_true_df = total_rev_geo_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_true_df[col] = total_rev_geo_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,TOL,TVS
0,202501,2025-05-28 02:04:38,0,"7,662,694,892","7,662,694,892","2,440,530,082","3,336,770,490","1,592,511,720","292,882,600"
1,202502,2025-05-28 02:04:38,0,"7,443,112,557","7,443,112,557","2,211,389,140","3,333,288,223","1,605,419,142","293,016,052"
2,202503,2025-05-28 02:04:38,0,"7,688,149,447","7,688,149,447","2,459,164,272","3,319,489,222","1,612,256,745","297,239,208"
3,202504,2025-05-28 02:04:38,0,"7,647,166,017","7,647,166,017","2,359,595,474","3,371,023,854","1,612,742,089","303,804,601"
4,202505,2025-05-28 02:04:38,0,"6,316,659,166","6,316,659,166","2,029,638,601","2,675,453,980","1,434,529,527","177,037,058"


In [14]:
''' DB0R000100GEO : Total Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000100GEO' #Total Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_dtac_df = total_rev_geo_dtac_df.loc[total_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_geo_dtac_df = total_rev_geo_dtac_df.loc[total_rev_geo_dtac_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_geo_dtac_df['TOTAL_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB0R000100GEO', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['PRE_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['POST_B2C_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['CHK_SUM'] = np.where(total_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), total_rev_geo_dtac_df['P'], 0)

total_rev_geo_dtac_df = total_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
total_rev_geo_dtac_df['CHK_DIFF'] = total_rev_geo_dtac_df['TOTAL_D'] - total_rev_geo_dtac_df['CHK_SUM']
total_rev_geo_dtac_df = total_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_dtac_df = total_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = total_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_dtac_df[col] = total_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D
0,202501,2025-05-28 02:04:38,0,"4,371,652,537","4,371,652,537","1,840,026,982","2,531,625,556"
1,202502,2025-05-28 02:04:38,0,"4,176,008,505","4,176,008,505","1,648,818,151","2,527,190,353"
2,202503,2025-05-28 02:04:38,0,"4,307,831,229","4,307,831,229","1,775,434,050","2,532,397,179"
3,202504,2025-05-28 02:04:38,0,"4,244,419,190","4,244,419,190","1,713,499,280","2,530,919,910"
4,202505,2025-05-28 02:04:38,0,"3,551,565,025","3,551,565,025","1,496,666,317","2,054,898,708"


In [15]:
''' B0R000101GEO : Mobile Revenue (Geo) '''

v_metric_list = [
    'B0R000101GEO' #Mobile Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_df = mobile_rev_geo_df.loc[mobile_rev_geo_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_geo_df = mobile_rev_geo_df.loc[mobile_rev_geo_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_geo_df['MOBILE'] = np.where(mobile_rev_geo_df['METRIC_CD']=='B0R000101GEO', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['CHK_SUM'] = np.where(mobile_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100']), mobile_rev_geo_df['P'], 0)

mobile_rev_geo_df = mobile_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_df['CHK_DIFF'] = mobile_rev_geo_df['MOBILE'] - mobile_rev_geo_df['CHK_SUM']
mobile_rev_geo_df = mobile_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_df = mobile_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_df[col] = mobile_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D
0,202501,2025-05-28 02:04:38,0,"10,148,953,110","10,148,953,110","2,440,530,082","1,840,026,982","3,336,770,490","2,531,625,556"
1,202502,2025-05-28 02:04:38,0,"9,720,685,868","9,720,685,868","2,211,389,140","1,648,818,151","3,333,288,223","2,527,190,353"
2,202503,2025-05-28 02:04:38,0,"10,086,484,724","10,086,484,724","2,459,164,272","1,775,434,050","3,319,489,222","2,532,397,179"
3,202504,2025-05-28 02:04:38,0,"9,975,038,517","9,975,038,517","2,359,595,474","1,713,499,280","3,371,023,854","2,530,919,910"
4,202505,2025-05-28 02:04:38,-0,"8,256,657,606","8,256,657,606","2,029,638,601","1,496,666,317","2,675,453,980","2,054,898,708"


In [16]:
''' TB0R000101GEO : Mobile Revenue (Geo) : TMH '''

v_metric_list = [
    'TB0R000101GEO' #Mobile Revenue (Geo) : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_geo_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.loc[mobile_rev_geo_tmh_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.loc[mobile_rev_geo_tmh_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_geo_tmh_df['MOBILE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB0R000101GEO', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['PRE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['POST_B2C_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['CHK_SUM'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R010100']), mobile_rev_geo_tmh_df['P'], 0)

mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_tmh_df['CHK_DIFF'] = mobile_rev_geo_tmh_df['MOBILE_T'] - mobile_rev_geo_tmh_df['CHK_SUM']
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_B2C_T']]

mod_col_list = mobile_rev_geo_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_tmh_df[col] = mobile_rev_geo_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_B2C_T
0,202501,2025-05-28 02:04:38,0,"5,777,300,573","5,777,300,573","2,440,530,082","3,336,770,490"
1,202502,2025-05-28 02:04:38,0,"5,544,677,363","5,544,677,363","2,211,389,140","3,333,288,223"
2,202503,2025-05-28 02:04:38,0,"5,778,653,494","5,778,653,494","2,459,164,272","3,319,489,222"
3,202504,2025-05-28 02:04:38,0,"5,730,619,327","5,730,619,327","2,359,595,474","3,371,023,854"
4,202505,2025-05-28 02:04:38,0,"4,705,092,581","4,705,092,581","2,029,638,601","2,675,453,980"


In [17]:
''' DB0R000101GEO : Mobile Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000101GEO' #Mobile Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.loc[mobile_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.loc[mobile_rev_geo_dtac_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_geo_dtac_df['MOBILE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB0R000101GEO', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['PRE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['POST_B2C_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['CHK_SUM'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), mobile_rev_geo_dtac_df['P'], 0)

mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_dtac_df['CHK_DIFF'] = mobile_rev_geo_dtac_df['MOBILE_D'] - mobile_rev_geo_dtac_df['CHK_SUM']
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_dtac_df[col] = mobile_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_B2C_D
0,202501,2025-05-28 02:04:38,0,"4,371,652,537","4,371,652,537","1,840,026,982","2,531,625,556"
1,202502,2025-05-28 02:04:38,0,"4,176,008,505","4,176,008,505","1,648,818,151","2,527,190,353"
2,202503,2025-05-28 02:04:38,0,"4,307,831,229","4,307,831,229","1,775,434,050","2,532,397,179"
3,202504,2025-05-28 02:04:38,0,"4,244,419,190","4,244,419,190","1,713,499,280","2,530,919,910"
4,202505,2025-05-28 02:04:38,0,"3,551,565,025","3,551,565,025","1,496,666,317","2,054,898,708"


### Revenue by Products

#### All Service

In [18]:
''' All Service '''

v_product_grp = 'All Services'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

all_service_df = monthly_df_display.copy()
all_service_df = all_service_df.loc[all_service_df['PRODUCT_GRP']==v_product_grp]
all_service_df = all_service_df.loc[all_service_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_df = all_service_df.reset_index(drop=True)
all_service_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,All Services,B0R000100,Total Revenue,2025-05-27 23:15:00,"7,405,956,874","10,318,463,203",0,0,0,"-2,912,506,329"
1,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-28 02:04:38,0,"9,868,224,191","9,686,506,881","9,686,255,178","9,686,271,965","-9,868,224,191"
2,202505,All Services,DB0R000100,Total Revenue : DTAC,2025-05-27 23:15:00,"3,735,853,188","3,735,858,488",0,0,0,"-5,299"
3,202505,All Services,DB0R000100GEO,Total Revenue (Geo) : DTAC,2025-05-28 02:04:38,0,"3,551,565,025","3,551,564,173","3,551,479,738","3,551,479,738","-3,551,565,025"
4,202505,All Services,TB0R000100,Total Revenue : TRUE,2025-05-27 23:15:00,"3,670,103,686","6,582,604,715",0,0,0,"-2,912,501,030"
5,202505,All Services,TB0R000100GEO,Total Revenue (Geo) : TRUE,2025-05-28 02:04:38,0,"6,316,659,166","6,134,942,708","6,134,775,440","6,134,792,227","-6,316,659,166"


#### Mobile

In [19]:
''' Mobile '''

v_product_grp = 'Mobile'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_df = monthly_df_display.copy()
mobile_df = mobile_df.loc[mobile_df['PRODUCT_GRP']==v_product_grp]
mobile_df = mobile_df.loc[mobile_df['TM_KEY_MTH']==v_tm_key_mth]
mobile_df = mobile_df.reset_index(drop=True)
mobile_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Mobile,B0R000101,Mobile Revenue,2025-05-27 23:15:00,"5,778,464,465","8,706,896,618",0,0,0,"-2,928,432,153"
1,202505,Mobile,B0R000101GEO,Mobile Revenue (Geo),2025-05-28 02:04:38,0,"8,256,657,606","8,074,940,296","8,074,329,686","8,074,329,686","-8,256,657,606"
2,202505,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-27 23:15:00,"3,735,853,188","3,735,858,488",0,0,0,"-5,299"
3,202505,Mobile,DB0R000101GEO,Mobile Revenue (Geo) : DTAC,2025-05-28 02:04:38,0,"3,551,565,025","3,551,564,173","3,551,479,738","3,551,479,738","-3,551,565,025"
4,202505,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-27 23:15:00,"2,042,611,277","4,971,038,130",0,0,0,"-2,928,426,853"
5,202505,Mobile,TB0R000101GEO,Mobile Revenue (Geo) : TMH,2025-05-28 02:04:38,0,"4,705,092,581","4,523,376,123","4,522,849,949","4,522,849,949","-4,705,092,581"


#### Prepaid

In [20]:
''' Prepaid '''

v_product_grp = 'Prepaid'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

prepaid_df = monthly_df_display.copy()
prepaid_df = prepaid_df.loc[prepaid_df['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Prepaid,B1R000100,Prepaid Revenue,2025-05-28 02:04:38,"3,539,272,294","3,526,304,918","3,344,596,335","3,343,996,480","3,343,996,480","12,967,377"
1,202505,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-05-28 02:04:38,"1,496,661,018","1,496,666,317","1,496,666,317","1,496,590,183","1,496,590,183","-5,299"
2,202505,Prepaid,TB1R000100,Prepaid Revenue : TMH,2025-05-28 02:04:38,"2,042,611,277","2,029,638,601","1,847,930,018","1,847,406,297","1,847,406,297","12,972,676"


#### Postpaid

In [21]:
''' Postpaid '''

v_product_grp = 'Postpaid'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

postpaid_df = monthly_df_display.copy()
postpaid_df = postpaid_df.loc[postpaid_df['PRODUCT_GRP']==v_product_grp]
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_df = postpaid_df.reset_index(drop=True)
postpaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Postpaid,B2R000100,Postpaid Revenue,2025-05-27 23:15:00,"5,091,314,947","5,091,314,947",0,0,0,0
1,202505,Postpaid,B2R010100,Postpaid Revenue B2C,2025-05-28 02:04:38,"4,642,946,619","4,642,946,619","4,642,938,090","4,642,927,696","4,642,927,696",0
2,202505,Postpaid,B2R020100,Postpaid Revenue B2B,2025-05-27 23:15:00,"448,368,328","448,368,328",0,0,0,0
3,202505,Postpaid,DB2R000100,Postpaid Revenue : DTAC,2025-05-28 02:04:38,"2,239,192,171","2,239,192,171","2,239,191,319","2,238,209,693","2,238,209,693",0
4,202505,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-05-28 02:04:38,"2,054,898,708","2,054,898,708","2,054,897,857","2,054,889,555","2,054,889,555",0
5,202505,Postpaid,DB2R020100,Postpaid Revenue B2B : DTAC,2025-05-28 02:04:38,"184,293,463","184,293,463","184,293,463","183,320,138","183,320,138",0
6,202505,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-05-27 23:15:00,"2,941,399,529","2,941,399,529",0,0,0,0
7,202505,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-28 02:04:38,"2,675,453,980","2,675,453,980","2,675,446,104","2,675,443,652","2,675,443,652",0
8,202505,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-05-27 23:15:00,"265,945,549","265,945,549",0,0,0,0


#### TOL

In [22]:
''' TOL '''

v_product_grp = 'TOL'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

tol_df = monthly_df_display.copy()
tol_df = tol_df.loc[tol_df['PRODUCT_GRP']==v_product_grp]
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==v_tm_key_mth]
tol_df = tol_df.reset_index(drop=True)
tol_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,TOL,TB3R000100,TOL Revenue,2025-05-28 02:04:38,"1,439,967,291","1,434,529,527","1,434,529,527","1,434,529,527","1,434,529,527","5,437,764"


#### TVS

In [23]:
''' TVS '''

v_product_grp = 'TVS'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

tvs_df = monthly_df_display.copy()
tvs_df = tvs_df.loc[tvs_df['PRODUCT_GRP']==v_product_grp]
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_df = tvs_df.reset_index(drop=True)
tvs_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,TVS,TB4R000100,TVS Revenue,2025-05-28 02:04:38,"187,525,117","177,037,058","177,037,058","177,395,964","177,412,751","10,488,060"
